# RNN 简介

# 时间序列模型概念简介
循环神经网络（RNN）是一种神经网络类型，其神经元的输出在下一个时间步会反馈作为输入，使网络具有处理序列数据的能力。它能处理变长序列，挖掘数据中的时序信息，不过存在长期依赖问题，即难以处理长序列中相距较远的信息关联。
RNN与普通神经网络的主要区别在于其具有记忆功能，神经元的输出能作为下一步输入，可处理序列数据，且输入和输出长度不固定；普通神经网络一般处理独立同分布的数据，层与层之间是简单的前馈连接关系，输入输出的长度通常是固定的。

RNN的应用场景广泛，在自然语言处理方面，可用于语言模型来预测下一个单词的概率，还能完成机器翻译、文本生成任务；在语音识别领域，能够处理语音这种时间序列信号，提高识别准确率；在时间序列预测中，像股票价格预测、天气预测等，RNN通过学习历史数据模式预测未来值；在视频分析中，它可以处理视频帧序列，进行动作识别等操作。





# RNN网络结构图
![图1](images/rnn.png)

RNN公式：
![rnn_rule](images/rnn_rule.png)

# 观察torch.nn.RNN的输入输出

In [1]:
import torch
import torch.nn as nn

In [2]:
# 单向、单层rnn
single_rnn = nn.RNN(input_size=4, hidden_size=3, num_layers=1, batch_first=True) # batch_first=True表示输入数据的维度为[batch_size, seq_len, input_size]
input = torch.randn(1, 5, 4) # 输入数据维度为[batch_size, seq_len, input_size]
output, h_n = single_rnn(input) # output维度为[batch_size, seq_len, hidden_size=3]，h_n维度为[num_layers=1, batch_size, hidden_size=3]
print(output, output.shape, h_n, h_n.shape,  sep='\n')

tensor([[[ 0.6290,  0.0402, -0.5033],
         [ 0.0400, -0.9701, -0.4599],
         [-0.3543,  0.5235, -0.8333],
         [-0.8115, -0.8594,  0.3722],
         [-0.7049, -0.7313,  0.5524]]], grad_fn=<TransposeBackward1>)
torch.Size([1, 5, 3])
tensor([[[-0.7049, -0.7313,  0.5524]]], grad_fn=<StackBackward0>)
torch.Size([1, 1, 3])


In [3]:
output[:, 2, :] 

tensor([[-0.3543,  0.5235, -0.8333]], grad_fn=<SliceBackward0>)

In [4]:
# 双向、单层rnn
bi_rnn = nn.RNN(input_size=4, hidden_size=3, num_layers=1, batch_first=True, bidirectional=True)
bi_output, bi_h_n = bi_rnn(input)
print(bi_output, bi_output.shape, bi_h_n, bi_h_n.shape, sep='\n')

tensor([[[-0.9372,  0.0044,  0.1953,  0.5167, -0.8242, -0.9832],
         [-0.4407,  0.8089, -0.0052,  0.9857, -0.6660, -0.7070],
         [-0.8628, -0.7891,  0.0801,  0.8280, -0.8767,  0.7452],
         [-0.4347,  0.7187, -0.8896,  0.9714,  0.7405,  0.8707],
         [-0.5838, -0.3942, -0.0465,  0.9107,  0.6390,  0.6951]]],
       grad_fn=<TransposeBackward1>)
torch.Size([1, 5, 6])
tensor([[[-0.5838, -0.3942, -0.0465]],

        [[ 0.5167, -0.8242, -0.9832]]], grad_fn=<StackBackward0>)
torch.Size([2, 1, 3])


# 从零手搓 RNN 

## forword

In [5]:
import torch
import torch.nn as nn

In [21]:
batch_size, seq_len, input_size, hidden_size = 2, 3, 2, 3 # 批次大小、序列长度、输入维度、隐藏层维度
num_layers = 1 # rnn层数

input = torch.randn(batch_size, seq_len, input_size) # 初始化输入数据
h_prev = torch.zeros(batch_size, hidden_size) # 初始化隐藏层状态

In [22]:
class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CustomRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.W_ih = nn.Parameter(torch.rand(self.input_size, self.hidden_size))
        self.W_hh = nn.Parameter(torch.rand(self.hidden_size, self.hidden_size))
        self.b_ih = nn.Parameter(torch.zeros(self.hidden_size))
        self.b_hh = nn.Parameter(torch.zeros(self.hidden_size))
        self.fc = nn.Linear(self.hidden_size, output_size)

    def forward(self, x, h_prev):
        print(x.size())
        batch_size, seq_length, _ = x.size()
        hiddens = []
        for t in range(seq_length):
            x_t = x[:, t, :]
            h_t = torch.tanh(torch.mm(x_t, self.W_ih) + self.b_ih + torch.mm(h_prev, self.W_hh) + self.b_hh)
            hiddens.append(h_t)
            h_prev = h_t
        h_final = hiddens[-1]
        output = self.fc(h_final)
        return output, h_prev

    def init_hidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)




## 训练

### 数据预处理
- 读取数据集：
  首先需要获取周杰伦的歌词数据集，可以从网络上搜索整理其歌词文本，将所有歌词保存到一个文本文件中，如jaychou_lyrics.txt。使用 Python 的open()函数读取文件内容，并进行必要的字符编码转换 。
- 建立字符索引：
将歌词中的每个字符映射为一个从 0 开始的连续整数索引，构建字符到索引的字典char_to_idx以及索引到字符的字典idx_to_char。通过遍历歌词文本，找出所有不同的字符，然后为每个字符分配一个唯一的索引。同时，可以得到词典大小vocab_size，即不同字符的数量 。
- 数据采样:

  对处理后的数据进行采样，以便生成训练所需的小批量数据。常见的采样方式有随机采样和相邻采样两种 ：
- 随机采样：
  每次从数据中随机选择一定长度的连续字符序列作为一个样本，同时对应的下一个字符作为该样本的标签。例如，若设定时间步数为num_steps，则每次随机选取num_steps个连续字符作为输入样本，其后面的一个字符作为输出标签。
相邻采样：按照顺序依次选取连续的字符序列作为样本和标签，即第i个样本的输入是从i到i + num_steps - 1的字符序列，其标签则是从i + 1到i + num_steps的字符序列。

In [34]:

# 假设输入歌词维度、隐藏层维度、层数、输出维度等
input_size = 100
hidden_size = 256
num_layers = 1
output_size = 100
rnn = CustomRNN(input_size, hidden_size, num_layers, output_size)

# 模拟输入数据（实际要根据歌词进行词向量等转换），这里假设一批次2条数据，序列长度5，维度为input_size
x = torch.randn(2, 5, input_size)
optimizer = optim.Adam(rnn.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(10):
    h_prev = rnn.init_hidden(2)
    output, h_prev = rnn(x, h_prev)
    loss = criterion(output, torch.randn(2, output_size))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')


NameError: name 'CustomRNN' is not defined

### 模型训练
参数初始化：初始化模型的参数，如词嵌入维度embedding_dim、隐藏层维度hidden_dim等，并定义损失函数和优化器。例如，可以使用交叉熵损失函数nn.CrossEntropyLoss()和随机梯度下降优化器torch.optim.SGD() 。
训练循环：在训练循环中，按照设定的批次大小和采样方式获取训练数据，将数据输入到模型中进行前向传播，计算损失值，然后使用优化器进行反向传播更新模型参数。在每个训练周期，可以打印出当前的损失值，以观察模型的训练进度 。

In [1]:
def train(model, data_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        hidden = None
        total_loss = 0
        for batch_x, batch_y in data_loader:
            optimizer.zero_grad()
            output, hidden = model(batch_x, hidden)
            loss = criterion(output, batch_y.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()
            total_loss += loss.item()
            hidden = hidden.detach()
        print(f'Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}')

### 模型测试与效果评估
- 生成歌词：训练完成后，可以使用训练好的模型来生成周杰伦风格的歌词。给定一个起始字符或字符序列，通过模型预测下一个可能的字符，然后将预测的字符作为新的输入，继续预测下一个字符，以此类推，生成一段歌词 。

In [2]:
def generate_text(model, char_to_idx, idx_to_char, start_text, length):
    model.eval()
    with torch.no_grad():
        input_text = torch.tensor([char_to_idx[char] for char in start_text]).unsqueeze(0)
        hidden = None
        generated_text = start_text
        for _ in range(length):
            output, hidden = model(input_text, hidden)
            output_probs = torch.softmax(output, dim=1)
            top_prob, top_idx = torch.topk(output_probs, k=1)
            top_char = idx_to_char[top_idx.item()]
            generated_text += top_char
            input_text = torch.tensor([top_idx]).unsqueeze(0)
        return generated_text

- 效果评估：可以从多个角度评估生成歌词的效果，如歌词的通顺性、连贯性、是否符合周杰伦的风格等。一种简单的方法是人工观察和评价生成的歌词，判断其是否具有一定的合理性和艺术感。也可以使用一些自动评估指标，如困惑度（Perplexity）等来定量地评估模型的性能，但困惑度指标并非完全能够准确反映生成文本的质量，仅供参考.

In [3]:
def calculate_perplexity(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_count = 0
    with torch.no_grad():
        for batch_x, batch_y in data_loader:
            output, _ = model(batch_x, None)
            loss = criterion(output, batch_y.view(-1))
            total_loss += loss.item() * batch_y.numel()
            total_count += batch_y.numel()
    return torch.exp(torch.tensor(total_loss / total_count))

通过以上步骤，就可以利用周杰伦的歌词训练 PyTorch RNN 模型，并对生成歌词的效果进行测试和评估 。需要注意的是，由于歌词的生成具有一定的主观性和创造性，模型的表现可能会因多种因素而有所不同，可通过调整模型结构、参数、训练数据等方式来进一步优化模型的性能 。

In [ ]:
s